In [1]:
from ctypes import *
import time
from sys import path
import sys
from os import sep
import numpy as np

if sys.platform.startswith("win"):
    dwf = cdll.dwf
    constants_path = "C:" + sep + "Program Files (x86)" + sep + "Digilent" + sep + "WaveFormsSDK" + sep + "samples" + sep + "py"
elif sys.platform.startswith("darwin"):
    dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
    constants_path = "/Applications/WaveForms.app/Contents/Resources/SDK/samples/py"
else:
    dwf = cdll.LoadLibrary("libdwf.so")
    constants_path = "/usr/share/digilent/waveforms/samples/py"

path.append(constants_path)
import dwfconstants as constants

In [2]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0)) #setting VDD = 3V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +3.00V


In [3]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [4]:
write_register(ENABLE_REG, 0x01) #Power on / PON = 1
time.sleep(0.01)
write_register(ENABLE_REG, 0x03) #Now, also RGBC enable / PON = 1, AEN = 1
write_register(ATIME_REG, 0xF6) #To run for 24ms, upto a max count of 10,240
write_register(CONTROL_REG, 0x02) #Setting AGAIN=16

True

In [5]:
#Function to test the sensor output
def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [10]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)

In [11]:
cleanup()
#call it everytime after work is done

In [8]:
write_register(CONTROL_REG,0)
for i in range(256):
    write_register(ATIME_REG,255-i)
    colors = read_colors()

    print(f"Clear: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

Clear: 6226, R: 1723, G: 9, B: 8
Clear: 51, R: 15, G: 18, B: 16
Clear: 74, R: 21, G: 30, B: 27
Clear: 100, R: 27, G: 39, B: 34
Clear: 124, R: 34, G: 49, B: 43
Clear: 148, R: 41, G: 61, B: 53
Clear: 180, R: 49, G: 70, B: 60
Clear: 202, R: 56, G: 80, B: 69
Clear: 227, R: 63, G: 90, B: 76
Clear: 249, R: 69, G: 100, B: 86
Clear: 278, R: 77, G: 110, B: 92
Clear: 298, R: 82, G: 118, B: 101
Clear: 325, R: 90, G: 129, B: 110
Clear: 353, R: 96, G: 137, B: 118
Clear: 366, R: 101, G: 144, B: 123
Clear: 393, R: 109, G: 155, B: 132
Clear: 386, R: 107, G: 152, B: 130
Clear: 116, R: 31, G: 46, B: 39
Clear: 79, R: 18, G: 26, B: 22
Clear: 10, R: 4, G: 0, B: 0
Clear: 1, R: 1, G: 0, B: 0
Clear: 1, R: 1, G: 0, B: 0
Clear: 2, R: 2, G: 0, B: 0
Clear: 2, R: 2, G: 0, B: 0
Clear: 2, R: 2, G: 0, B: 0
Clear: 2, R: 2, G: 0, B: 1
Clear: 2, R: 2, G: 0, B: 1
Clear: 2, R: 2, G: 0, B: 1
Clear: 2, R: 2, G: 1, B: 1
Clear: 2, R: 2, G: 1, B: 1
Clear: 2, R: 2, G: 1, B: 1
Clear: 2, R: 2, G: 1, B: 1
Clear: 3, R: 2, G: 1, B: 

KeyboardInterrupt: 

In [11]:
print("Settings applied. Stabilizing for 1 second...")
time.sleep(1.0) 
print("Stabilizing done, now testing...")
print(" ")

clear_channel_samples = []
for i in range(1000):
    colors = read_colors()
    
    if colors:
        # Save the 'Clear' channel for statistical analysis
        clear_channel_samples.append(colors['clear'])
        
        # Print every 100th sample to avoid flooding the output window
        if i % 100 == 0:
            print(f"Sample {i}: Clear: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

#ANALYSIS
if len(clear_channel_samples) > 0:
    mean_val = np.mean(clear_channel_samples)
    std_dev = np.std(clear_channel_samples)
    noise_percent = (std_dev / mean_val) * 100
    
    print("\n" + "="*40)
    print(f"ANALYSIS RESULT (N=1000)")
    print(f"Mean Count:       {mean_val:.2f}")
    print(f"Standard Dev:     {std_dev:.2f}")
    print(f"Noise (Jitter):   {noise_percent:.3f}%")
    print("="*40)
else:
    print("Error: No data collected.")

Settings applied. Stabilizing for 1 second...
Stabilizing done, now testing...
 
Sample 0: Clear: 4117, R: 1179, G: 1658, B: 1380
Sample 100: Clear: 4048, R: 1166, G: 1631, B: 1355
Sample 200: Clear: 3997, R: 1157, G: 1611, B: 1334
Sample 300: Clear: 4008, R: 1159, G: 1616, B: 1339
Sample 400: Clear: 3998, R: 1157, G: 1612, B: 1335
Sample 500: Clear: 4023, R: 1163, G: 1621, B: 1344
Sample 600: Clear: 4051, R: 1169, G: 1632, B: 1355
Sample 700: Clear: 4051, R: 1169, G: 1632, B: 1356
Sample 800: Clear: 4049, R: 1168, G: 1631, B: 1355
Sample 900: Clear: 4029, R: 1163, G: 1623, B: 1347

ANALYSIS RESULT (N=1000)
Mean Count:       4029.75
Standard Dev:     23.39
Noise (Jitter):   0.580%
